In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
data = np.genfromtxt('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv',  delimiter = ',',  skip_header = 1)
print(data.shape)

(1097, 477)


In [3]:
# 1
transposed_data = data.T # transpose and make one row for each movie
print(transposed_data.shape)

ratings = transposed_data[:400]
colNum = data.shape

popularity = []
for movie in ratings:
    popularity.append(np.isfinite(movie).sum())

median = np.median(np.array(popularity))
print('median: ', median)

low = []
high = []
for movie in ratings:
    if np.isfinite(movie).sum() < median:
        low.append(movie)
    else:
        high.append(movie)

low_medians = []
for movie in low:
    low_medians.append(np.nanmedian(movie))

high_medians = []
for movie in high:
    high_medians.append(np.nanmedian(movie))
    
u1,p1 = stats.mannwhitneyu(high_medians, low_medians, alternative = 'greater')
print(p1)

(477, 1097)
median:  197.5
9.929258851707232e-35


In [46]:
# 2
# data_with_name = np.genfromtxt('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', delimiter = ',', names = True)
# movie_titles = data_with_name.dtype.names[:400]
# movie_titles

#find median
data_with_name = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
titles = data_with_name.columns

years = []
titles = titles[:400]
print(len(titles))
for title in titles:
    if title.rfind('(') == -1:
        continue
    
    #print(title)
    start = title.rfind('(')
    end = title.rfind(')')
    year_str = title[start+1:end]
    year_str = year_str.strip()
    years.append(int(year_str))
    #year = int(float(year_str))
    
median = np.median(np.array(years))
print('median: ', median)

#data_with_name = data_with_name[:400]
data_with_name = data_with_name.iloc[:, 0:400]
print(data_with_name.shape)


low_medians = []
high_medians = []
for title in data_with_name.columns:
    if title.rfind('(') == -1:
        continue
        
    start = title.rfind('(')
    end = title.rfind(')')
    year_str = title[start+1:end]
    year_str = year_str.strip()
    year = int(year_str)
    
    if year > median:
        rating = data_with_name[title].to_numpy()
        high_medians.append(np.nanmedian(rating))
#         print(year)
#         print(type(data_with_name[title].to_numpy())
    else:
        rating = data_with_name[title].to_numpy()
        low_medians.append(np.nanmedian(rating))
        
u1,p1 = stats.mannwhitneyu(high_medians, low_medians)
print(p1)



# # find median
# data_with_name = np.genfromtxt('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', delimiter = ',', names = True)

# #data_with_name = np.genfromtxt('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', delimiter = ',')
# print(data_with_name.shape)
# print(data_with_name)
# movie_titles = data_with_name.dtype.names[:400]

# years = []
# for title in movie_titles:
#     start = title.rfind('_')
#     year = title[start+1:]
#     #if year.isnumeric():
#         #print(int(year))
    
    

400
median:  1999.0
(1097, 400)
0.21113405794975193


In [47]:
# 3 TODO check self-rated?
data = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
shrek = data['Shrek (2001)'].to_numpy()
gender = data['Gender identity (1 = female; 2 = male; 3 = self-described)'].to_numpy()
#len(gender) == len(shrek)

female_rates = []
male_rates = []
for i in range(0, len(gender)):
    if gender[i] == 1 and not np.isnan(shrek[i]):
        female_rates.append(shrek[i])
    elif gender[i] == 2 and not np.isnan(shrek[i]):
        male_rates.append(shrek[i])
        
print(len(female_rates), len(male_rates))

u1,p1 = stats.mannwhitneyu(female_rates, male_rates) # alternative: two-sided
print(p1)
    

743 241
0.050536625925559006


In [48]:
# 4
data = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
titles = data_with_name.columns
titles = titles[:400]
gender = data['Gender identity (1 = female; 2 = male; 3 = self-described)'].to_numpy()

count = 0
for title in titles:
    movie_rating = data[title]
    
    female_rates = []
    male_rates = []
    for i in range(0, len(gender)):
        if gender[i] == 1 and not np.isnan(movie_rating[i]):
            female_rates.append(movie_rating[i])
        elif gender[i] == 2 and not np.isnan(movie_rating[i]):
            male_rates.append(movie_rating[i])
            
    u1, p1 = stats.mannwhitneyu(female_rates, male_rates) # alternative: two-sided
    
    if p1 < 0.005:
        count += 1

print(count)
count / 400
    
    

50


0.125

In [49]:
# 5
data = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
lion_king = data['The Lion King (1994)'].to_numpy()
relation = data['Are you an only child? (1: Yes; 0: No; -1: Did not respond)'].to_numpy()

only_children = []
siblings = []

for i in range(0, len(gender)):
    if relation[i] == 1 and not np.isnan(lion_king[i]):
        only_children.append(lion_king[i])
    elif relation[i] == 0 and not np.isnan(lion_king[i]):
        siblings.append(lion_king[i])
        
u1,p1 = stats.mannwhitneyu(only_children, siblings, alternative = 'greater')
print(p1)

0.978419092554931


In [50]:
# 6
data = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
titles = data_with_name.columns
titles = titles[:400]
relation = data['Are you an only child? (1: Yes; 0: No; -1: Did not respond)'].to_numpy()

count = 0
for title in titles:
    movie_rating = data[title]
    
    only_children = []
    siblings = []
    for i in range(0, len(relation)):
        if relation[i] == 1 and not np.isnan(movie_rating[i]):
            only_children.append(movie_rating[i])
        elif relation[i] == 0 and not np.isnan(movie_rating[i]):
            siblings.append(movie_rating[i])
            
    u1, p1 = stats.mannwhitneyu(siblings, only_children)
    
    #print(p1)
    if p1 < 0.005:
        count += 1

print(count)
count / 400

7


0.0175

In [59]:
# 7
data = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
titles = data_with_name.columns
titles = titles[:400]
wolf_of_wall_street = data['The Wolf of Wall Street (2013)'].to_numpy()
socially_enjoy = data['Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)'].to_numpy()

movies_socially = []
movies_alone = []

for i in range(0, len(socially_enjoy)):
    if socially_enjoy[i] == 1 and not np.isnan(wolf_of_wall_street[i]):
        movies_alone.append(wolf_of_wall_street[i])
    elif socially_enjoy[i] == 0 and not np.isnan(wolf_of_wall_street[i]):
        movies_socially.append(wolf_of_wall_street[i])
        
print(len(movies_socially), len(movies_alone))        
u1, p1 = stats.mannwhitneyu(movies_socially, movies_alone, alternative = 'greater')
print(p1)

270 393
0.9436657996253056


In [60]:
# 8
data = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
titles = data_with_name.columns
titles = titles[:400]
socially_enjoy = data['Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)'].to_numpy()

count = 0
for title in titles:
    movie_rating = data[title]
    
    movies_socially = []
    movies_alone = []
    
    for i in range(0, len(socially_enjoy)):
        if socially_enjoy[i] == 1 and not np.isnan(movie_rating[i]):
            movies_alone.append(movie_rating[i])
        elif socially_enjoy[i] == 0 and not np.isnan(movie_rating[i]):
            movies_socially.append(movie_rating[i])
    
    u1, p1 = stats.mannwhitneyu(movies_socially, movies_alone) # alternative = two-sided
    
    if p1 < 0.005:
        count += 1

print(count)
count / 400

10


0.025

In [69]:
# 9
data = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
ha = data['Home Alone (1990)'].to_numpy()
fn = data['Finding Nemo (2003)'].to_numpy()

len(ha[~np.isnan(ha)])
len(fn[~np.isnan(fn)])

haa = ha[~np.isnan(ha)]
fnn = fn[~np.isnan(fn)]

stats.ks_2samp(haa, fnn) # default: two-sided

KstestResult(statistic=0.15269080020897632, pvalue=6.379381467525036e-10)

In [103]:
# 10
data = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
franchises = ['Star Wars', 'Harry Potter', 'The Matrix', 'Indiana Jones', 'Jurassic Park', 'Pirates of the Caribbean', 'Toy Stroy', 'Batman']

# do 1
title = 'Star Wars'
series = data.loc[:, data.columns.str.contains(title)]
#num_row, num_col = series.shape
#num_col

dropped = series.dropna()
splitted = []
for dropped_movie in dropped:
    dropped_np = dropped[dropped_movie].to_numpy()
    splitted.append(dropped_np)
    
stats.kruskal(splitted[0], splitted[1], splitted[2], splitted[3], splitted[4], splitted[5])    

    
#n_arr[~np.isnan(n_arr).any(axis=1)]

KruskalResult(statistic=193.51026675400544, pvalue=6.940162236984522e-40)

In [17]:
data = pd.read_csv('/Users/yjeonlee/Desktop/[DS-GA-1001]Intro to Data Science/Project1/movieReplicationSet.csv', skipinitialspace = True)
data.head()

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II,Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1


In [18]:
# 1
ratings = data.iloc[:, 0:400]
popularity = {} # key: movie name, value: the number of ratings

for key in ratings.columns:
    popularity[key] = ratings[key].replace('', np.nan).count()
    #print(key)
    #print(ratings[key].replace('', np.nan).count())
    
dict(sorted(popularity.items(), key = lambda item: item[1]))

#find median
popularity_lst = []
for key in popularity:
    popularity_lst.append(popularity[key])
    
popularity_lst = sorted(popularity_lst)
#median = popularity_lst[len(popularity_lst)//2]

print("median: ", (popularity_lst[199] + popularity_lst[200])/2)

low = []
high = []

#print(ratings.columns.str.contains('a'))
for key in popularity:
    if popularity[key] < median:
        low.append(ratings[key])
    else:
        high.append(ratings[key])
        
np.var(popularity_lst)


u1,p1 = stats.mannwhitneyu(low,high)

#low = low[:len(low)-2]
#low
        
      #low.append(ratings.loc[:, ratings.columns.str.contains(key)])
    #high.append(ratings.loc[:, ratings.columns.str.contains(key)])

# low = pd.DataFrame() 
# high = pd.DataFrame()
# for key in popularity:
#     if popularity[key] < median:
#         low.append(pd.DataFrame(ratings.loc[:, ratings.columns.str.contains(key)]))
#     else:
#         high.append(pd.DataFrame(ratings.loc[:, ratings.columns.str.contains(key)]))
        
# # low

# #pd.DataFrame.from_dict(popularity)
# converted_pd = pd.DataFrame([popularity], columns=popularity.keys())
# converted_np = converted_pd.to_numpy()
# converted_pd.median()
# #df = pd.DataFrame([d], columns=d.keys())


median:  197.5


ValueError: `x` and `y` must not contain NaNs.

In [ ]:
for key in popularity:
    print(key, ' : ', popularity[key])